In [1]:
import numpy as np

기본적인 RNN 계층 구현

In [ ]:
class RNN:
  def __init__(self, Wx, Wh, b):
    self.params = [Wx, Wh, b]
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
    self.cache = None

  # RNN의 순전파 계산
  def forward(self, x, h_prev):
    Wx, Wh, b = self.params

    t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
    h_next = np.tanh(t)

    self.cache = (x, h_prev, h_next)
    return h_next

  # RNN의 역전파 계산
  def backward(self, dh_next):
    Wx, Wh, b = self.params
    x, h_prev, h_next = self.cache

    dt = dh_next * (1 - h_next ** 2)
    db = np.sum(dt, axis=0)
    dWh = np.matmul(h_prev.T, dt)
    dh_prev = np.matmul(dt, Wh.T)
    dWx = np.matmul(x.T, dt)
    dx = np.matmul(dt, Wx.T)

    self.grads[0][...] = dWx
    self.grads[1][...] = dWh
    self.grads[2][...] = db

    return dx, dh_prev

앞서 구현한 RNN 계층들을 모아서 처리하는 Time RNN을 구현해본다. Time RNN 계층은 RNN 계층 T개를 연결한 신경망이다.

In [ ]:
class TimeRNN:
  def __init__(self, Wx, Wh, b, stateful=False):
    self.params = [Wx, Wh, b]
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)] # 저장할 미분값들을 0으로 초기화
    self.layers = None # 다수의 RNN 계층을 리스트로 저장하는 용도

    self.h, self.dh = None, None
    self.stateful = stateful

  def set_state(self, h):
    self.h = h

  def reset_state(self):
    self.h = None

  # Time RNN 순전파 계산
  def forward(self, xs):
    Wx, Wh, b = self.params
    N, T, D = xs.shape # N:미니배치 크기, T:시계열 데이터의 분량, D:입력벡터의 차원 수
    D, H = Wx.shape # D:입력벡터의 차원 수, H:은닉 벡터의 차원 수

    self.layers = []
    hs = np.empty((N, T, H), dtype='f') # 출력값을 담을 float형 그릇, N:미니배치 크기, T:시계열 데이터의 분량, H:은닉 벡터의 차원 수

    if not self.stateful or self.h is None: # 처음 호출 시에 RNN계층의 은닉상태 h를 0행렬로 초기화
      self.h = np.zeros((N, H), dtype='f')

    for t in range(T):
      layer = RNN(*self.params) # 앞서 구현했던 RNN 계층을 이용
      self.h = layer.forward(xs[:, t, :], self.h) # t번째 입력에 대해서 RNN 계층 순전파 수행
      hs[:, t, :] = self.h  # 각 시각 t의 출력(은닉 상태)을 저장
      self.layers.append(layer) #생성하여 사용된 RNN 계층을 laysers 리스트에 추가하여 저장

    return hs

  # Time RNN 역전파
  def backward(self, dhs):
    Wx, Wh, b = self.params
    N, T, H = dhs.shape # N:미니배치 크기, T:시계열 데이터의 분량, H:은닉 벡터의 차원 수
    D, H = Wx.shape # D:입력벡터의 차원 수, H:은닉 벡터의 차원 수

    dxs = np.empty((N, T, D), dtype='f') # 역전파 입력
    dh = 0  # 역전파를 거치며 합산된 기울기
    grads = [0, 0, 0] # 미분값 초기화

    for t in reversed(range(T)): # 역순서의 인덱스로 순회
      layer = self.layeres[t] # RNN 계층을 역으로 순회
      dx, dh = layer.backward(dhs[:, t, :] + dh) # 역전파 수행 (순전파에서 분기되었으므로 역전파에서는 기울기를 합산, 분기의 반대는 합산이다!)
      dxs[:, t, :] = dx # 각 시각 t에서 얻은 x 기울기를 저장

      # t를 순회할 때마다 역전파로 얻은 가중치를 위 로컬변수로 초기화 한 grads에 합산
      for i, grad in enumerate(layer.grads):
        grads[i] += grad

    # 합산된 가중치 기울기의 최종 결과를 멤버변수 self.grads에 덮어씀
    for i, grad in enumerate(grads):
      self.grads[i][...] = grad
    self.dh = dh

    return dxs